In [11]:
using  Revise, Rasters, GeoFormatTypes, GeoTIFF, ArchGDAL
using Statistics

include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")
include("Julia//GeoNetcdf.jl")
include("Julia//PlotParameter.jl")
include("Julia//GeoPlot.jl")

Path_Home = raw"C:\OSGeo4W\Gis\FABDEM\IRELAND_MOSAIC"
Path_Output = raw"C:\OSGeo4W\Gis\FABDEM\IRELAND_OUTPUT"
Files =readdir(Path_Home)


26-element Vector{String}:
 "N51W008_FABDEM_V1-0.tif"
 "N51W009_FABDEM_V1-0.tif"
 "N51W010_FABDEM_V1-0.tif"
 "N51W011_FABDEM_V1-0.tif"
 "N52W006_FABDEM_V1-0.tif"
 "N52W007_FABDEM_V1-0.tif"
 "N52W008_FABDEM_V1-0.tif"
 "N52W009_FABDEM_V1-0.tif"
 "N52W010_FABDEM_V1-0.tif"
 "N52W011_FABDEM_V1-0.tif"
 ⋮
 "N54W007_FABDEM_V1-0.tif"
 "N54W008_FABDEM_V1-0.tif"
 "N54W009_FABDEM_V1-0.tif"
 "N54W010_FABDEM_V1-0.tif"
 "N54W011_FABDEM_V1-0.tif"
 "N55W006_FABDEM_V1-0.tif"
 "N55W007_FABDEM_V1-0.tif"
 "N55W008_FABDEM_V1-0.tif"
 "N55W009_FABDEM_V1-0.tif"

In [ ]:
Maps = []
for iiFile in Files
    Dem_Map₀ = Rasters.Raster(joinpath(Path_Home, iiFile))
    Dem_Map  = Rasters.replace_missing(Dem_Map₀, missingval=NaN)
    Maps     = push!(Maps, Dem_Map)
end


In [13]:
Mosaic = Rasters.mosaic(first, Maps; missingval=NaN, progress=true)


Mosaicing 26 regions with first... 100%|██████████████████████████████████████████████████| Time: 0:00:00


┌ 21600×18000 Raster{Float64, 2} ┐
├────────────────────────────────┴─────────────────────────────────────── dims ┐
  ↓ X Projected{Float64} -11.000208333333333:0.0002777777777777778:-5.000486111111111 ForwardOrdered Regular Intervals{Start},
  → Y Projected{Float64} 55.999861111111116:-0.0002777777777777785:51.000138888888884 ReverseOrdered Regular Intervals{Start}
├────────────────────────────────────────────────────────────────────── raster ┤
  missingval: NaN
  extent: Extent(X = (-11.000208333333333, -5.000208333333333), Y = (51.000138888888884, 56.00013888888889))
  crs: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.25722...
└──────────────────────────────────────────────────────────────────────────────┘
   ↓ →       55.9999   55.9996   55.9993  …   51.0007   51.0004   51.0001
 -11.0002   NaN       NaN       NaN            0.0       0.0       0.0
 -10.9999   NaN       NaN       NaN            0.0       0.0       0.0
 -10.9997   NaN       NaN       NaN            

In [14]:
Param_Crs_GeoFormat = GeoFormatTypes.convert(WellKnownText, EPSG(Param_Crs))

Mosaic_Resample = Rasters.resample(Mosaic; res=ΔX₂, method=ResampleMethod₂, crs=Param_Crs_GeoFormat)
#  Mosaic_Resample = Rasters.replace_missing(Mosaic_Resample, missingval=NaN)


┌ 21055×28024 Raster{Float64, 2} ┐
├────────────────────────────────┴─────────────────────────────────────── dims ┐
  ↓ X Projected{Float64} -10566.616338923981:20.0:410513.383661076 ForwardOrdered Regular Intervals{Start},
  → Y Projected{Float64} 532331.122395666:-20.0:-28128.877604334033 ReverseOrdered Regular Intervals{Start}
├──────────────────────────────────────────────────────────────────── metadata ┤
  Metadata{Rasters.GDALsource} of Dict{String, Any} with 1 entry:
  "filepath" => "/vsimem/tmp"
├────────────────────────────────────────────────────────────────────── raster ┤
  missingval: NaN
  extent: Extent(X = (-10566.616338923981, 410533.383661076), Y = (-28128.877604334033, 532351.122395666))
  crs: PROJCS["TM65 / Irish Grid",GEOGCS["TM65",DATUM["TM65",SPHEROID["Airy...
└──────────────────────────────────────────────────────────────────────────────┘
      ↓ →          5.32331e5    5.32311e5  …  -28088.9  -28108.9  -28128.9
 -10566.6        NaN          NaN                N

In [ ]:
include("Julia//GeoRaster.jl")

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Mosaic_Resample; Verbose=true)

	Longitude, Latitude = Rasters.X(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right, crs=Metadatas.Crs_GeoFormat), Rasters.Y(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom, crs=Metadatas.Crs_GeoFormat)

	Mosaic_Resample, Mosaic_Boarder₀ = geoRaster.DEM_CORRECT_BOARDERS!(; Dem=Mosaic_Resample, Latitude, Longitude, Crs=Metadatas.Crs_GeoFormat, iiParam_GaugeCoordinate=Param_GaugeCoordinate)

	Mosaic_Boarder = Rasters.replace_missing(Mosaic_Boarder₀, missingval=NaN)


In [ ]:
	Path_OutputJulia_Dem = joinpath(Path_Output, "BOARDERS_Ireland_FABDEM.tiff")
	Rasters.write(Path_OutputJulia_Dem, Mosaic_Boarder; ext=".tiff", missingval= NaN, force=true, verbose=true)


In [ ]:

# Writting the output
	Path_OutputJulia_Dem = joinpath(Path_Output, "DEM_Ireland_FABDEM2.tiff")
	Rasters.write(Path_OutputJulia_Dem, Mosaic_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)


In [ ]:
# geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Mosaic, Title=string("iiFile"), Label="True [m]")
# using GLMakie
# fig = GLMakie.Figure()
# ga = Axis(
#     fig[1, 1] # any cell of the figure's layout
#     # dest = "+proj=wintri", # the CRS in which you want to plot
# )
# # lines!(Maps, GeoMakie.coastlines()) # plot coastlines from Natural Earth as a reference
# # # You can plot your data the same way you would in Makie
# GLMakie.heatmap!(ga, Mosaic)
# fig
